# **Data preprocessing**

This notebook consists of the data preprocessing steps used for preparing the harvested Twitter data using twarc2 into nicely formatted data usefull for further analyzing.

In general, the data preprocessing was done following the next workflow:

1.   Setup the Google Colab environment
2.   Load the data
3.   Filter the data
  *   Drop Tweets without known sender location
  *   Drop one-way conversations
  *   Add the location of receivers 
  *   Drop Tweets without known receiver location  
4.   Export the data







## Setup the Google Colab environment


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/gdrive')

# Import pandas
import pandas as pd
# Used to interact with directories
import os

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Load the data

We read the harvested Tweets from multiple files into a single Pandas DataFrame. The data is harvested using the [twarc2](https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/) api for 3 consecutive weeks, where we filtered on the Dutch word "ik". 

The first week we harvested 632994, the second week 606552, and the third week 594251 Tweets.



In [ ]:
# Takes all .csv files in a directory and write it into a single Pandas DataFrame
directory_path= "/gdrive/MyDrive/TwitterData_GroupB/Data/Tweets/" 

# Create empty DataFrame to fill
full_tweets_df = pd.DataFrame()

# iterate over specified directory
for filename in os.listdir(directory_path):
    # set file path
    file_path = os.path.join(directory_path,filename)
    if os.path.isfile(file_path):
      # check if file is a .csv file
      if file_path.endswith('.csv'):
        print(f"Loading: {file_path}.")
        # Read csv into a Pandas DataFrame
        tweets = pd.read_csv(file_path, encoding='utf-8', engine='python') # Specified encoding and engine to not get a DtypeWarning
        # Concatenate tweets from single file to collective Tweets DataFrame
        full_tweets_df = pd.concat([full_tweets_df, tweets])

full_tweets_df.info()


Loading: /gdrive/MyDrive/TwitterData_GroupB/Data/Tweets/dutchTweets1.csv.
Loading: /gdrive/MyDrive/TwitterData_GroupB/Data/Tweets/dutchTweets2.csv.
Loading: /gdrive/MyDrive/TwitterData_GroupB/Data/Tweets/dutchTweets3.csv.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1833797 entries, 0 to 594250
Data columns (total 83 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   id                                     int64  
 1   conversation_id                        int64  
 2   referenced_tweets.replied_to.id        float64
 3   referenced_tweets.retweeted.id         float64
 4   referenced_tweets.quoted.id            float64
 5   author_id                              int64  
 6   in_reply_to_user_id                    float64
 7   in_reply_to_username                   object 
 8   retweeted_user_id                      float64
 9   retweeted_username                     object 
 10  quoted_user_id                       

For privacy concerns, we drop all the columns that we won't need for our analysis.

In [ ]:
# Define columns to keep
column_list = ['id', 'author_id', 'text', 'conversation_id', 'in_reply_to_user_id', 'geo.geo.bbox']

# Filter DataFrame using list of columns
filtered_tweets_df = full_tweets_df.filter(column_list, axis=1)

# Convert column datatype to nullable integer and change column name
filtered_tweets_df['in_reply_to_user_id'] = filtered_tweets_df[['id', 'author_id', 'conversation_id', 'in_reply_to_user_id']].astype('Int64')
filtered_tweets_df.rename(columns={'geo.geo.bbox': 'sender_location'}, inplace=True)

filtered_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1833797 entries, 0 to 594250
Data columns (total 6 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   id                   int64 
 1   author_id            int64 
 2   text                 object
 3   conversation_id      int64 
 4   in_reply_to_user_id  Int64 
 5   sender_location      object
dtypes: Int64(1), int64(3), object(2)
memory usage: 99.7+ MB


## Filter the data

Firstly, since we are only interested in Tweets which have a location, we filter out all Tweets that are not geotagged.

In [ ]:
print(f"Shape before deleting non-geotagged Tweets: {filtered_tweets_df.shape}.")

# Dropping All Tweets without geolocation
geo_tweets = filtered_tweets_df.dropna(subset=['sender_location']).copy()

# We need geo_tweets for a later step. We copy geo_tweets into a new dataframe tweets_df for further filtering
tweets_df = geo_tweets

# Compare shapes
print(f"Shape after deleting non-geotagged Tweets: {tweets_df.shape}.")


Shape before deleting non-geotagged Tweets: (1833797, 6).
Shape after deleting non-geotagged Tweets: (30242, 6).
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30242 entries, 14 to 594151
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   30242 non-null  int64 
 1   author_id            30242 non-null  int64 
 2   text                 30242 non-null  object
 3   conversation_id      30242 non-null  int64 
 4   in_reply_to_user_id  22512 non-null  Int64 
 5   sender_location      30242 non-null  object
dtypes: Int64(1), int64(3), object(2)
memory usage: 1.6+ MB


Since we are interested in the sentiments in Twitter conversations, we only want to keep Tweets that are part of a conversation with other users to avoid one-way conversation. 


In [ ]:
# Drop Tweets that are not part of a conversation with 2 or more Tweets
tweets_df = tweets_df.groupby('conversation_id').filter(lambda x: len(x)>=2)
print(f"Shape after deleting tweets not part of a conversation: {tweets_df.shape}.")

# Drop Tweets of users who reply to themselves
tweets_df = tweets_df[tweets_df['in_reply_to_user_id'] != tweets_df['author_id']]
print(f"Shape after deleting self-replies: {tweets_df.shape}.")


Shape after deleting tweets not part of a conversation: (10576, 6).
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10576 entries, 74 to 594151
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   10576 non-null  int64 
 1   author_id            10576 non-null  int64 
 2   text                 10576 non-null  object
 3   conversation_id      10576 non-null  int64 
 4   in_reply_to_user_id  9716 non-null   Int64 
 5   sender_location      10576 non-null  object
dtypes: Int64(1), int64(3), object(2)
memory usage: 588.7+ KB


Next we need to prepare the geometries needed for the network analysis. We start by including the receiver location to every Tweet.


In [ ]:
def addReceiverLocation(df1, df2):
  '''This function takes two dataframes, adds the receiver location of a tweet by 
  matching the in_reply_to_user_id to the corrosponding author_id in a larger dataframe containing all the geotagged data before filtering,
  and returns a dataframe with added receiver location.  '''
    # Keep unique author_id's
    df2e = df2.drop_duplicates(subset=['author_id'])
    # Create empty list to store receiver locations
    receiverLocations = []
    # Loop through rows in dataframe
    for index1, row1 in df1.iterrows():
        # Get receiver id
        receiverID = row1['in_reply_to_user_id']
        # Get receiver location
        recieverLocation = []
        for index2, row2 in df2e.iterrows():
            # Check if the receiver id is equal to the author id in df2
            if receiverID == row2['author_id']:\
                # If condition is met, set receiver location as the corrosponding sender location
                recieverLocation = row2['sender_location']
                break
        # Append Receiver Location to List
        receiverLocations.append(recieverLocation)
    # Add Receiver Locations to Dataframe
    df1['receiver_location'] = receiverLocations
    return df1

Running this function will take over 40 minutes since it is not optimized at the moment. After runninig this function we exported a .csv file, so there is no need for rerunning the code. For now we will comment out the next code block, and load the cvs file we exported. Only un-comment the next code block if you want to check if both output are the same.

In [ ]:
# Add receiver location to the dataframe
#tweets_df_with_receiver_loc = addReceiverLocation(tweets_df, geo_tweets)

#tweets_df_with_receiver_loc.info()

(9198, 7)
1797


,id,author_id,text,conversation_id,in_reply_to_user_id,sender_location,receiver_location
74,1641772946434797569,177542012,@TotallyAnders Dat vermoed ik ook. Of het is z...,1641013257153421312,2168925260,"[5.5257694, 53.0666403, 5.7656546, 53.1837484]",[]
1328,1641769755773157379,123950495,@SaschaJanssen @merchandise @JaapFriso @MerelM...,1641442842055938049,46652559,"[5.7576521, 51.7905544, 5.9082602, 51.8947409]",[]
1492,1641769330424659969,135445960,@Eneco @EnecoNieuws Ik kan jullie niet dm stur...,1641719488549732353,177189845,"[4.1903465, 52.0148713, 4.4224441, 52.1388646]",[]
1536,1641769213462302721,21298074,@mirjamlederwerg Ik geloof het wel.,1641455728958357506,454142767,"[-4.7764112, 36.4863005, -4.5921218, 36.6255465]","[4.7946271, 52.1615468, 5.0217418, 52.3041119]"
1589,1641769076153368578,21298074,@mirjamlederwerg Die ken ik hier niet.,1641720811701653504,454142767,"[-4.7764112, 36.4863005, -4.5921218, 36.6255465]","[4.7946271, 52.1615468, 5.0217418, 52.3041119]"


The next code block is a safestate for when we want to do something with the tweets after the receiver_location gets added, we don't have to rerun the function that adds this column, since otherwise it will take over 40 minutes.

In [ ]:
# Read .csv file into DataFrame
tweets_df_with_receiver_loc = pd.read_csv("/gdrive/MyDrive/TwitterData_GroupB/Data/Preprocessed/tweets.csv")
# Define columns to keep
column_list = ['id', 'author_id', 'text', 'conversation_id', 'in_reply_to_user_id', 'sender_location', 'receiver_location']

# Filter DataFrame using list of columns
tweets_df_with_receiver_loc = tweets_df_with_receiver_loc.filter(column_list, axis=1)

tweets_df_with_receiver_loc.info()

,id,author_id,text,conversation_id,in_reply_to_user_id,sender_location,receiver_location
0,1641769213462302721,21298074,@mirjamlederwerg Ik geloof het wel.,1641455728958357506,454142767,"[-4.7764112, 36.4863005, -4.5921218, 36.6255465]","[4.7946271, 52.1615468, 5.0217418, 52.3041119]"
1,1641769076153368578,21298074,@mirjamlederwerg Die ken ik hier niet.,1641720811701653504,454142767,"[-4.7764112, 36.4863005, -4.5921218, 36.6255465]","[4.7946271, 52.1615468, 5.0217418, 52.3041119]"
2,1641765800250843136,143788349,@MisterSenseo @MilfHummie Heel stom! Als ik we...,1641526240451371031,365677125,"[4.6750992, 51.9056092, 4.8323339, 51.9841161]","[4.1903465, 52.0148713, 4.4224441, 52.1388646]"
3,1641760234489167873,123950495,@merchandise @SaschaJanssen @JaapFriso @MerelM...,1641442842055938049,7167342,"[5.7576521, 51.7905544, 5.9082602, 51.8947409]","[5.1830769, 51.3055699, 5.3137105, 51.4365525]"
4,1641756738385854465,940899377543110656,@Pieterjanvl Maar ik was dus Vlaanderen aan he...,1641726576650772481,48287280,"[5.2121062, 50.8677279, 5.4145591, 50.9815265]","[4.774005, 50.9533704, 4.9369701, 51.0376519]"


Lastly, we filter out tweets where there is an empty receiver_location. Tweets can have no receiver_location of the receiver has no known location in our harvested data set.

In [ ]:
# Filter out tweets that have no receiver_location
tweets_df_with_receiver_loc_filtered = tweets_df_with_receiver_loc[tweets_df_with_receiver_loc.receiver_location != "[]"]

print(f"Shape after deleting Tweets with no receiver location: {tweets_df_with_receiver_loc_filtered.shape}.")


(1051, 7)

## Export the results

We are done with filtering the Tweets. For further processing using Natural Language Processing, the id and text of a Tweet will be exported since other columns are not needed for this analysis step. For the geospatial analysis, the full dataframe will be exported to .csv file. 

In [ ]:
# Write id and text columns to .csv file for further processing using NLP
tweets_df_with_receiver_loc_filtered[['id', 'text']].to_csv('/gdrive/MyDrive/TwitterData_GroupB/Data/Preprocessed/tweets_text.csv', index=False)

# Write dataframe to .csv file for doing thhe geospatial analysis
tweets_df_with_receiver_loc_filtered.to_csv('/gdrive/MyDrive/TwitterData_GroupB/Data/Preprocessed/tweets.csv', index=False)

## Discussion

Since the preprocessing involved some difficult decisions, there are also some drawbacks to some of the decisions that we made. First, only geotagged tweets were used. This led to a reduction of 98% of the initial dataset and therefore the dataset used for the spatial analysis is not representative for the whole dataset or Dutch social media users. However, given the size of the preprocessed dataset (1051 conversation pairs), deriving a conclusion is still possible. In addition, the methodology of this project provides proof of concept.

Tweets that were part of an conversation_id that only occurred once were filtered out. This was done to avoid one-way conversations, and thus only take into account bidirectional tweets. However, it could be possible that tweets without the "ik" keyword were present in the conversation, but not in our dataset. This limitation is a result of our data harvesting method as some interactions are lost.

Our method to establish a receiver’s location comes with some limitations. Our approach was to look for the presence of the in_reply_to_user_id from the resulting dataframe and match them to the user_id in the older dataframe with all geotagged tweets. Since people can also tweet from locations other than their home (vacation location, job office, etc.), we are aware that this location may not necessarily be their residency. Of course, this is true for all location data. However, for receiver’s locations there is an extra limitation due to the assumption that the first corresponding match is the receiver’s location for all occurrences of this in_reply_to_user_id.